In [114]:
import pandas as pd
import requests
import json
import re
import numpy as np

In [79]:
post_data = {
    "limit": 2000
}

In [80]:
r = requests.post("http://13.126.59.19:20021/api/search/VendorProducts",data=json.dumps(post_data))

In [81]:
print(r.status_code, r.reason)

200 OK


In [82]:
result = json.loads(r.text)

In [83]:
df = pd.io.json.json_normalize(result['products'])

In [84]:
columns_to_use = ['locationAddress.city', 'locationAddress.state', 'locationAddress.country', 'locationName',
                  'product.color', 'product.grade', 'product.group', 'product.variety', 'productOffer.boxType',
                  'productOffer.height', 'productOffer.length', 'productOffer.price.amount',
                  'productOffer.quantityPerBox', 'vendorName']

In [98]:
data_df = df[columns_to_use]

In [99]:
def tokenize_on_caps(x):
    x = ' '.join(re.findall('[A-Z,(1-9)][^A-Z]*', x))
    return re.sub('[^A-Za-z0-9 ]+', '', x.strip())

In [100]:
data_df.loc[:,'product.color'] = data_df['product.color'].apply(lambda x: tokenize_on_caps(x))
data_df.loc[:,'productOffer.price.amount'] = data_df['productOffer.price.amount'].apply(lambda x: round(x,2))

/home/ganesh/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [101]:
data_df

,locationAddress.city,locationAddress.state,locationAddress.country,locationName,product.color,product.grade,product.group,product.variety,productOffer.boxType,productOffer.height,productOffer.length,productOffer.price.amount,productOffer.quantityPerBox,vendorName
0,Bogota,Bogota,Columbia,Exotic Farms SAS,Lilac Pink,Alesso,Supplies,Solfarino,FB,10,10,0.39,400,Tulipan Flowers
1,Bogota,Bogota,Columbia,Exotic Farms SAS,Fall,8-11 Bloom,Pre-made bouquets,Fifth Avenue,FB,10,10,0.12,300,Tulipan Flowers
2,Miami,Florida,United States,Polo Farms,Lavender Purple,8-10 cm,Cut Greens,With All My Heart,FB,10,10,0.95,100,Polo Farms
3,Bogota,Bogota,Columbia,Mounika Bogota,Antique,18 Gram,Fresh flowers,Red Velvet Henriette,FB,10,10,0.95,100,Mounika Farms
4,Bogota,Bogota,Columbia,Exotic Farms SAS,Purple Blue,Monte Lava,Collateral materials,Catwalk,FB,10,10,0.39,100,Tulipan Flowers
5,Miami,Florida,United States,Sande Miami,Varigated,10 St,Plants,Verena,FB,10,10,0.38,200,Calla Flowers By Sande
6,Miami,Holland,Netherlands,Premium Dutch Mia,Light Green,Extra,Collateral materials,Takiny,FB,10,10,0.18,100,Tuning
7,Miami,Holland,Netherlands,Premium Dutch Mia,Green Yellow,6.5 Inch,Plants,Amina,FB,10,10,0.95,200,Tuning
8,Quito,Quito,Ecuador,Bellaflor Group-Special,Green Antique,Titanium,Plants,Preziosa,FB,10,10,0.79,100,Exportadora de Flores
9,Miami,Florida,United States,Polo Farms,Solid Purple,Chico,Fresh flowers,Carousel,FB,10,10,0.72,100,Polo Farms


In [102]:
categorical_cols = ['locationAddress.city', 'locationAddress.state', 'locationAddress.country', 'locationName',
                   'product.color', 'product.grade', 'product.group', 'product.variety', 'productOffer.boxType',
                   'vendorName']

In [103]:
upcoming_cats = {'locationAddress.city':20, 'locationAddress.state':10, 
                 'locationAddress.country':5, 'locationName':20,
                 'product.color':25, 'product.grade':25, 'product.group':30,
                 'product.variety':30, 'productOffer.boxType':10,
                 'vendorName':30
                }

In [104]:
from sklearn.preprocessing import LabelBinarizer 

In [105]:
def get_encoders(df, cols, extra_fields=None):
    encoders = dict()
    for column in cols:
        label_binarizer = LabelBinarizer()
        cats = df[column].unique().tolist() + ['other_'+str(i) for i in range(upcoming_cats[column])]
        label_binarizer.fit(cats)
        encoders.setdefault(column, (label_binarizer.fit(cats), cats))
    return encoders

In [106]:
def transform_df(df, cols, extra_fields):
    encoders = get_encoders(df, cols, extra_fields)
    for column in cols:
        one_hot = encoders[column][0].transform(df[column])
        new_df = pd.DataFrame(data=one_hot, columns=[column+'_'+name for name in encoders[column][1]])
        df.drop(labels=column, axis=1, inplace=True)
        df = pd.concat([df, new_df], axis=1)
    return df

In [107]:
new_df = transform_df(data_df, categorical_cols, upcoming_cats)

/home/ganesh/miniconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [108]:
new_df

,productOffer.height,productOffer.length,productOffer.price.amount,productOffer.quantityPerBox,locationAddress.city_Bogota,locationAddress.city_Miami,locationAddress.city_Quito,locationAddress.city_other_0,locationAddress.city_other_1,locationAddress.city_other_2,...,vendorName_other_20,vendorName_other_21,vendorName_other_22,vendorName_other_23,vendorName_other_24,vendorName_other_25,vendorName_other_26,vendorName_other_27,vendorName_other_28,vendorName_other_29
0,10,10,0.39,400,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10,10,0.12,300,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10,10,0.95,100,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10,10,0.95,100,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10,10,0.39,100,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,10,10,0.38,200,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,10,10,0.18,100,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,10,10,0.95,200,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,10,10,0.79,100,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,10,10,0.72,100,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [109]:
from sklearn.metrics.pairwise import cosine_similarity

In [111]:
similarity_mat = cosine_similarity(new_df)

In [112]:
def get_similars(df, indices):
    for index in reversed(indices):
        print(df.iloc[index].values)

In [119]:
sims = np.argsort(similarity_mat[0])[-6:]

In [120]:
get_similars(data_df, sims)

['Bogota' 'Columbia' 'Exotic Farms SAS' 'Lilac Pink' 'Alesso' 'Supplies'
 'Solfarino' 'FB' 10 10 0.39 400 'Tulipan Flowers']
['Bogota' 'Columbia' 'Exotic Farms SAS' 'Dark Bronze' 'Van Gogh Petite'
 'Supplies' 'Lesotho' 'FB' 10 10 0.38 400 'Tulipan Flowers']
['Bogota' 'Columbia' 'Exotic Farms SAS' 'Novelty Bicolor' '25 Lbs'
 'Supplies' 'Craspedia Globosa' 'FB' 10 10 0.6 400 'Tulipan Flowers']
['Bogota' 'Columbia' 'Exotic Farms SAS' 'White Yellow' 'Bouquet Grade'
 'Supplies' 'Chilis' 'FB' 10 10 0.6 400 'Tulipan Flowers']
['Bogota' 'Columbia' 'Exotic Farms SAS' 'White Burgundy Lip'
 '20 Stem Large' 'Supplies' 'Captain Melrose' 'FB' 10 10 0.6 400
 'Tulipan Flowers']
['Bogota' 'Columbia' 'Exotic Farms SAS' 'Blush' '1000 gms' 'Supplies'
 'Purple Crane' 'FB' 10 10 0.6 400 'Tulipan Flowers']


In [ ]:
['Bogota' 'Bogota' 'Columbia' 'Exotic Farms SAS' 'Pink Blue' 'Mega'
 'Supplies' 'Ecroland' 'FB' 10 10 0.72 400 'Tulipan Flowers']
['Bogota' 'Bogota' 'Columbia' 'Exotic Farms SAS' 'White Dyed Red'
 'Ecuadorian' 'Supplies' 'Prince Of Darkness' 'FB' 10 10 0.18 300
 'Tulipan Flowers']
['Bogota' 'Bogota' 'Columbia' 'Exotic Farms SAS' 'Pink Purple' 'Mega'
 'Supplies' 'Chapeau' 'FB' 10 10 0.39 100 'Tulipan Flowers']
['Bogota' 'Bogota' 'Columbia' 'Exotic Farms SAS' 'Light Purple' 'Chico'
 'Supplies' 'Lemoncello' 'FB' 10 10 0.65 300 'Tulipan Flowers']
['Bogota' 'Bogota' 'Columbia' 'Exotic Farms SAS' 'Brown Orange' 'Alesso'
 'Collateral materials' 'A1' 'FB' 10 10 0.79 100 'Tulipan Flowers']